In [1]:
import tensorflow as tf

In [2]:
device_name = tf.test.gpu_device_name()
import torch
device = torch.device("cpu")

In [3]:
!pip install transformers

import pandas as pd

In [4]:
df = pd.read_csv("../train3_512_max_char.csv", encoding = 'unicode escape') #@param {type:"string"}

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

Number of training sentences: 6,240



In [5]:
# Get the lists of sentences and their labels.
text = df.text.values
labels = df.label.values

In [6]:
from transformers import BertTokenizer
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [7]:
max_len = 0

# For every sentence...
for sent in text:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(str(sent), add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)


Max sentence length:  221


In [8]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in text:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        str(sent),                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        truncation = True,
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

/usr/local/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1767: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', text[0])
print('Token IDs:', input_ids[0])

Original:    
  block the US from being stationed on the islands off Hokkaido in the strategic Sea of Okhotsk, if this helps persuade Russia to give them back. 
The islands are inhabited, and in Russia are called the Southern Kurils â but for Japan they are Etorofu, Kunashiri, Shikotan, and the Habomai islet group. These territories, which became Russian after Japanâs defeat in World War II under the San Francisco P
Token IDs: tensor([  101,  3796,  1996,  2149,  2013,  2108,  8895,  2006,  1996,  3470,
         2125, 20826,  1999,  1996,  6143,  2712,  1997,  7929, 12326,  6711,
         1010,  2065,  2023,  7126, 13984,  3607,  2000,  2507,  2068,  2067,
         1012,  1996,  3470,  2024,  9613,  1010,  1998,  1999,  3607,  2024,
         2170,  1996,  2670, 13970, 15928,  2015,   100,  2021,  2005,  2900,
         2027,  2024,  3802, 14604, 11263,  1010, 28919, 12914,  3089,  1010,
        11895, 27380,  2078,   102])


In [10]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


5,616 training samples
  624 validation samples


In [11]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


In [12]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

import numpy as np

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [14]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

import random
import numpy as np

In [15]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()


In [16]:
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        print(np.shape(b_input_ids))
        b_input_mask = batch[1].to(device)
        print(np.shape(b_input_mask))
        b_labels = batch[2].to(device).long()
        print(np.shape(b_labels))

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        print(b_input_ids[0])
        print(b_input_mask[0])
        print(b_labels[0])
        
        print(torch._C._get_tracing_state())
        
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))


======== Epoch 1 / 2 ========
Training...
torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([32])
tensor([  101,  3745,  2006,  9130,  2009,  2323,  2022, 10358,  2065,   100,
         2206,  2739,  7175,  1996,  3061,  2600,  8090,  1999,  2167,  7734,
         2008,  6980,  4247,  2000,  9686, 25015,  2618,  2090,  6186,  5668,
         4637,  1996,  3061,  2600, 16615,  5917,  1998, 11421,  2610,  1012,
         1996,  2502,  3066,  1029,  1037,  2176,  1011,  2110,  7734,  3229,
        13117,  2029, 17016,  2000,  2039,  3217,  4140,  6730,  8940,  2598,
         1010,  9947,  1996,   102])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor(0)
None
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 183 of modeling_bert.py forward function BertEmbeddings

output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linea

I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 3

output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 320 of modeling_bert.py forward function BertSelfOutput.
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 383 of modeling_bert.py forward function BertIntermediate.
I a

output.shape-------------------------------------------
torch.Size([32, 64, 3072])
output and bias for linear shape of them..........
torch.Size([32, 64, 3072])
torch.Size([3072])
I am at line 532 of module.py.
I am at line 399 of modeling_bert.py forward function BertOutput.
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 3072])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output

output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 320 of modeling_bert.py forward function BertSelfOutput.
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 768])
output and bias for linear shape of them..........
torch.Size([32, 64, 768])
torch.Size([768])
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 532 of module.py.
I am at line 383 of modeling_bert.py forward function BertIntermediate.
I am at line 532 of module.py.
input size:
torch.Size([32, 64, 768])
output.shape-------------------------------------------
torch.Size([32, 64, 3072])
output and bias for linear shape of them..........
torch.Size([32, 64, 3072])
torch.Size([3072])


KeyboardInterrupt: 